In [34]:
import pandas as pd
import numpy as np
import copy 

from pandas_profiling import ProfileReport
from datetime import datetime, timedelta
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB 
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

def _encode(df, labels, label):
    """
        Sempre: 
            0-HODL
            1-BUY
            2-SELL
    """
    df[label] = 0
    df.loc[df[labels[0]] == 1, label] = 1
    df.loc[df[labels[1]] == 1, label] = 2
    return df
def _final_encode(df, labels, label):
    """
        Sempre: 
            0-HODL
            1-BUY
            2-SELL
        no caso do final, ação = 1; ver nos indicadores e aplicar a ação escolhida
    """
    df[label] = 0
    df[label] = df[labels].sum(axis=1)
    df[label] = [ 1 if x.astype(int) > 0 else 0 for x in df[label].values ]
    return df
def _filter_Train_n_Test(df):
    train = df.loc[df.Data < pd.to_datetime("01/02/2018", format="%d/%m/%Y")]
    test = df.loc[df.Data >= pd.to_datetime("01/02/2018", format="%d/%m/%Y")]
    return train, test
def _preprocess(df, X_labels, y_label):
    train, test = _filter_Train_n_Test(df)
    X_train = train[X_labels]
    y_train = train[y_label]
    X_test = test[X_labels]
    y_test = test[y_label]
    scaler = MinMaxScaler()
    # Prevenindo data leakage
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, y_train, X_test, y_test, scaler

def _apply_gaussianNB(df, features, label, silent=False):
    """
    Basea-se na suposição de que os dados seguem a distribuição normal para a predição da probabilidade a priori.
    As features devem ter valores contínuos (caso dos valores de preço!)
    """
    model = GaussianNB()
    X_train, y_train, X_test, y_test, scaler = _preprocess(df, features, label)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    if (silent):
        return model, scaler
    print(y_train.value_counts())
    print("Accuracy (Model: {} | Gaussian Naive Bayes):".format(label),metrics.accuracy_score(y_test, y_pred))
    return model, scaler

def _apply_complementNB(df, features, label, silent=False, _alpha = 1):
    """
    Seguimos a distribuição normal, mas generalizamos a predição calculando a probabilidade do item pertencer a todas as classes.
    (calculamos a probabilidade do item não pertencer a cada classe, selecionamos o menor valor, tendo em vista que calculamos a probabilidade de não ser da classe em cálculo)
    Útil para datasets desbalanceados!
    """
    model = ComplementNB(alpha=_alpha)
    X_train, y_train, X_test, y_test, scaler = _preprocess(df, features, label)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    if (silent):
        return model, scaler
    print(y_train.value_counts())
    print("Accuracy (Model: {} | Complement Naive Bayes):".format(label),metrics.accuracy_score(y_test, y_pred))
    return model, scaler

def _apply_multinomialNB(df, features, label, silent=False, _alpha = 1):
    """
    Esse algoritmo usa os dados em uma distribuição multinomial, que é uma generalização da distribuição binomial. 
    Essa distribuição é parametrizada por vetores θyi=(θy1,…,θyn), θyi é a probabilidade do evento i ocorrer, dado que a classe é y
    """
    model = MultinomialNB(alpha=_alpha)
    X_train, y_train, X_test, y_test, scaler = _preprocess(df, features, label)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    if (silent):
        return model, scaler
    print(y_train.value_counts())
    print("Accuracy (Model: {} | Multinomial Naive Bayes):".format(label),metrics.accuracy_score(y_test, y_pred))
    return model, scaler

def _apply_model(df, model, scaler, features, label, decisions):
    """
    Aplica as decisões dos primeiros modelos, para avaliação de como ponderar para uma gestão de risco baseada em resultados anteriores
    """
   # Variáveis de apoio
    trades = 0 if len(decisions)==0 else len(decisions) - 1

    # Ordenar DF pela data (mais antiga primeiro)
    df = df.sort_values('Data', ascending=True).reset_index(drop=True)

    # Separamos as colunas para a análise
    cols = copy.deepcopy(features)
    cols.append(label)
    cols.append('Data')

    # Pegamos do df as colunas
    features_df = df[cols]

    # para cada linha, aplicamos a decisão do modelo
    for i, item in df.iterrows():
        row_features = scaler.transform(pd.DataFrame(item[features]).T)
        _data = item['Data']
        if (trades > 0 and decisions.loc[trades-1, 'classifier'] == label):
            _last_decision_date = decisions.iloc[trades-1].Data
            _last_model_decision = decisions.iloc[trades-1].model_decision
            _last_real_decision = decisions.iloc[trades-1].real_decision
        else:
            _last_decision_date = pd.to_datetime('01/01/1950')
            _last_model_decision = 2
            _last_real_decision = 2
        _classifier = label
        _real_decision = item[label]
        _model_decision = model.predict(row_features)[0] if label != 'divergency_decisor' else item[label] # No caso do decisor de divergência vamos usar o próprio dado real
        _value_fechamento = item['Fechamento']
        #se a data atual for maior que a data do ultimo trade e decisao diferente de HOLD e da anterior:
        #if ((_model_decision != 0) or (_real_decision != 0)):
        if ((_data >= _last_decision_date + timedelta(days=wait_time))):
            if ((_model_decision != 0 and _model_decision != _last_model_decision) or (_real_decision != 0 and _real_decision != _last_real_decision)):
            # Adicionamos nas decisões os dados de data, classificador, decisao real, decisao do modelo e valor
                decisions.loc[trades,'Data'] = _data
                decisions.loc[trades,'classifier'] = _classifier
                decisions.loc[trades,'real_decision'] = _real_decision
                decisions.loc[trades,'model_decision'] = _model_decision
                decisions.loc[trades,'value_fechamento'] = _value_fechamento
                trades += 1
    return decisions

def _apply_decisions(model_decisions, classifier, money_start, decision_weight, n):
    """
    Aqui, aplicamos as decisões dos modelos sem uma gestão de risco, para gerar os pesos de ponderação para a gestão de risco;
    "All in, All out!", ou seja, compramos tudo ou vendemos tudo!
    """
    last_decision = 2
    money_atual = money_start
    quant_comprada = 0
    print("--------------------------------------------------------")
    print(classifier)
    print("--------------------------------------------------------")
    for i, row in model_decisions.iterrows():
        if (row.model_decision != last_decision and row.model_decision !=0):
            last_decision = row.model_decision
            if (last_decision == 1):
                quant_comprada = float(money_atual) / float(row.value_fechamento)
                money_atual = float(money_atual) - (float(quant_comprada) * float(row.value_fechamento))
                print("Decisão de compra -")
                print('Valor: {}'.format(str(row.value_fechamento)))
                print('Quantidade comprada: {}'.format(str(quant_comprada)))
                print("Money atual: {}".format(str(money_atual)))
            elif (last_decision == 2):
                money_comprado = float(quant_comprada) * float(row.value_fechamento)
                money_atual = float(money_atual) + float(money_comprado)
                print("Decisão de venda -")
                print('Valor: {}'.format(str(row.value_fechamento)))
                print('Quantidade vendida: {}'.format(str(quant_comprada)))
                print("Money atual: {}".format(str(money_atual)))
            else:
                print("HODL!")
        else:
            print("HODL!")
    if (last_decision == 1):
         money_atual = float(quant_comprada) * float(row.value_fechamento)
    decision_weight.loc[n, 'classifier'] = classifier
    decision_weight.loc[n, 'initial_maney'] = money_start
    decision_weight.loc[n, 'final_maney'] = money_atual
    decision_weight.loc[n, 'lucro_percentual'] = (money_atual - money_start)/money_start
    return decision_weight

def _apply_decisions_withperc(model_decisions, money_start, silent_mode=True):
    """
    Aplica as decisoes encontradas utilizando a ponderação dos melhores valores como método de gestão de risco
    """
    results = pd.DataFrame(columns = ['Data','initial_maney','final_maney','quant_hold','value_fechamento','perc_usado', 'action'])
    last_decision = 2
    money_atual = money_start
    quant_comprada = 0
    trades = 0
    for i, row in model_decisions.iterrows():
        if (row.real_decision != last_decision and row.real_decision !=0):
            data = row.Data
            last_decision = row.real_decision
            if (last_decision == 1):
                action='compra'
                money = float(money_atual) * float(row.perc_aplicado)
                quant_acomprar = (float(money) / float(row.value_fechamento))
                quant_comprada = quant_comprada + quant_acomprar
                money_atual = float(money_atual) - (money)
                if (silent_mode==False):
                    print("--------------------------")
                    print("Decisão de compra -")
                    print('Valor: {}'.format(str(row.value_fechamento)))
                    print('Quantidade comprada: {}'.format(str(quant_acomprar)))
                    print('Quantidade em hold: '+str(quant_comprada))
                    print("Money atual: {}".format(str(money_atual)))

            elif (last_decision == 2):
                action='venda'
                quant_vender = float(quant_comprada) * float(row.perc_aplicado)
                quant_comprada = float(quant_comprada) - float(quant_vender)
                money_comprado = float(quant_vender) * float(row.value_fechamento)
                money_atual = float(money_atual) + float(money_comprado)
                if (silent_mode==False):
                    print("--------------------------")
                    print("Decisão de venda -")
                    print('Valor: {}'.format(str(row.value_fechamento)))
                    print('Quantidade vendida: {}'.format(str(quant_vender)))
                    print('Quantidade em hold: '+str(quant_comprada))
                    print("Money atual: {}".format(str(money_atual)))
            else:
                if (silent_mode==False):
                    print("HODL!")
        else:
            if (silent_mode==False):
                print("HODL!")
        results.loc[trades, 'Data'] = data
        results.loc[trades, 'initial_maney'] = money_start
        results.loc[trades, 'final_maney'] = money_atual
        results.loc[trades, 'quant_hold'] = quant_comprada
        results.loc[trades, 'value_fechamento'] = float(row.value_fechamento)
        results.loc[trades, 'perc_usado'] = float(row.perc_aplicado)
        results.loc[trades, 'action'] = action
        trades +=1
    if (last_decision == 1 or quant_comprada > 0):
        money_atual = money_atual + (float(quant_comprada) * float(row.value_fechamento))
        quant_comprada = 0
        results.loc[trades, 'Data'] = data
        results.loc[trades, 'initial_maney'] = money_start
        results.loc[trades, 'final_maney'] = money_atual
        results.loc[trades, 'quant_hold'] = quant_comprada
        results.loc[trades, 'value_fechamento'] = float(row.value_fechamento)
        results.loc[trades, 'perc_usado'] = float(row.perc_aplicado)
        results.loc[trades, 'action'] = 'final'
    return results

def _get_model_decisions(df, classifiers, scalers, models, wait_time=5, silent_mode=True):
    """
    Aqui, utilizando os modelos encontrados:
        -verificamos quantos modelos consideram cada ação (buy, hold, sell),
        -caso todos consiredem hold, não faz nada;
        -caso a maioria seja buy/sell, compramos com a soma da ponderação encontrada a partir dos resultados anteriores
    """
    decisions = pd.DataFrame(columns=['Data','real_decision','value_fechamento','perc_aplicado'])
    # Variáveis de apoio
    trades = 0

    # Ordenar DF pela data (mais antiga primeiro)
    df = df.sort_values('Data', ascending=True).reset_index(drop=True)
    for i, item in df.iterrows():
        _buy = []
        _sell = []
        _hold = []
        _value_fechamento = item['Fechamento']
        for classifier in classifiers:
            row_features = scalers.get(
                classifier
            ).transform(
                pd.DataFrame(
                    item[features.get(classifier)]
                ).T
            )
            _model_decision = (models.get(classifier).predict(row_features)[0] if classifier != 'divergency_decisor' else item[classifier])
            if (_model_decision == 1):
                _buy.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
            elif (_model_decision == 2):
                _sell.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
            else:
                _hold.append(1)
        if (len(_hold) == 4):
            _model_decision = 0
            perc_ = 0
        elif (len(_buy) > len(_sell)):
            _model_decision = 1
            perc_ = sum(_buy)
            _data = item['Data']

        else:
            _model_decision = 2
            perc_ = sum(_sell)
            _data = item['Data']

        if (perc_ > 0):
            if (trades > 0):
                _last_decision_date = decisions.iloc[trades-1].Data
                _last_real_decision = decisions.iloc[trades-1].real_decision
            else:
                _last_decision_date = pd.to_datetime('01/01/1950')
                _last_real_decision = 2
            #se a data atual for maior que a data do ultimo trade e decisao diferente de HOLD e da anterior:
            #if ((_model_decision != 0) or (_real_decision != 0)):
            if ((_data >= _last_decision_date + timedelta(days=wait_time))):
                if ((_model_decision != 0 and _model_decision != _last_real_decision)):
                # Adicionamos nas decisões os dados de data, classificador, decisao real, decisao do modelo e valor
                    new_decision = {
                        'Data':_data,
                        'real_decision':_model_decision,
                        'value_fechamento':_value_fechamento,
                        'perc_aplicado':perc_}
                    decisions = decisions.append(new_decision, ignore_index=True)
                    trades += 1
    return decisions

def _get_final_decisions(df, _stop_loss, _sell_by_stoploss, wait_time, final_scaler, final_model, classifiers, scalers, models):
    """
        -Função final com modelo de trade!
        Iremos, utilizando os suportes como stop loss, e quando encontrar um preço de fechamento 
    igual ou menor ao suporte em um ponto de suporte, a venda de x% (definido pela var _sell_by_stoploss), aplicar os modelos encontrados quando o modelo final decidir agir.
    
    """
    # Variáveis de apoio
    trades = 0
    _last_decision_date = pd.to_datetime('01/01/1950')
    _last_real_decision = 2
    decisions = pd.DataFrame(columns=['Data','real_decision','value_fechamento','perc_aplicado','stop_loss','final_model', 'holds','buys','sells'])
    # Ordenar DF pela data (mais antiga primeiro)
    df = df.sort_values('Data', ascending=True).reset_index(drop=True)

    for i, item in df.iterrows():
        _buy = []
        _sell = []
        _hold = []
        final_hold = 1
        _value_fechamento = item['Fechamento']
        if (item['Suporte ']==1):
            _stop_loss = item['Mínimo']
        row_features = final_scaler.transform(pd.DataFrame(item[final_features]).T)
        _model_decision = final_model.predict(row_features)[0] ## Pegamos a decisão do modelo geral
        decisao_modelo_final = final_model.predict(row_features)[0]
        if (_model_decision == 1): ## se for uma decisão de ação,
            final_hold = 0
            for classifier in classifiers: ## aplicaremos os modelos criados anteriormente
                row_features = scalers.get(
                    classifier
                ).transform(
                    pd.DataFrame(
                        item[features.get(classifier)]
                    ).T
                )
                _model_decision = (models.get(classifier).predict(row_features)[0] if classifier != 'divergency_decisor' else item[classifier])
                if (_model_decision == 1):
                    _buy.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
                elif (_model_decision == 2):
                    _sell.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
                else:
                    _hold.append(decision_weight.loc[decision_weight.classifier == classifier,'new_weight'].values[0])
        else:
            final_hold = 1
            _model_decision = 0
            perc_ = 0
        if (decisao_modelo_final == 0): ## Usamos um comitê dos decisores: o mais votado, vira a ação! caso o modelo inicial seja hold, aplica:
            _model_decision = 0
            perc_ = 0
            _data = item['Data']
        elif (len(_buy) > len(_sell)):
            _model_decision = 1
            perc_ = sum(_buy)
            _data = item['Data']
        elif (len(_sell) > len(_buy)):
            _model_decision = 2
            perc_ = sum(_sell)
            _data = item['Data']
        else:
            _model_decision = 0
            perc_ = 0
            _data = item['Data']

        if (_model_decision > 0):
            if (trades > 0):
                _last_decision_date =  pd.to_datetime(decisions.tail(1).Data.values[0])
                _last_real_decision = decisions.tail(1).real_decision.values[0]
        #se a data atual for maior que a data do ultimo trade e decisao diferente de HOLD e da anterior:
        #if ((_model_decision != 0) or (_real_decision != 0)):
        if ((_data >= _last_decision_date + timedelta(days=wait_time))):
            if (_value_fechamento <= _stop_loss and item['Suporte ']==1): #se encontrarmos um preço de fechamento menor ou igual ao suporte e um ponto de suporte, vendemos x%
                _model_decision = 2
                perc_ = _sell_by_stoploss
                new_decision = {
                    'Data':_data,
                    'real_decision':_model_decision,
                    'value_fechamento':_value_fechamento,
                    'perc_aplicado':perc_,
                    'stop_loss':_stop_loss,
                    'holds':len(_hold),
                    'buys':len(_buy),
                    'sells':len(_sell),
                    'final_model':decisao_modelo_final}
                decisions = decisions.append(new_decision, ignore_index=True)
                trades += 1
            elif ((_model_decision != 0 and _model_decision != _last_real_decision)):
            # Adicionamos nas decisões os dados de data, classificador, decisao real, decisao do modelo e valor
                new_decision = {
                    'Data':_data,
                    'real_decision':_model_decision,
                    'value_fechamento':_value_fechamento,
                    'perc_aplicado':perc_,
                    'stop_loss':_stop_loss,
                    'holds':len(_hold),
                    'buys':len(_buy),
                    'sells':len(_sell),
                    'final_model':decisao_modelo_final}
                decisions = decisions.append(new_decision, ignore_index=True)
                trades += 1
    return decisions

def _print_decisions(decisions):
    """
    Função auxiliar para printar as decisões
    """
    print("Data inicial: {}".format(decisions.head(1).Data.values[0]))
    print("Dinheiro inicial: R${:.2f}".format(decisions.tail(1).initial_maney.values[0]))
    print("Data final: {}".format(decisions.tail(1).Data.values[0]))
    print("Dinheiro final: R${:.2f}".format(decisions.tail(1).final_maney.values[0]))
    print("Lucro final : {:.2f}%".format(((decisions.tail(1).final_maney.values[0] - decisions.tail(1).initial_maney.values[0])/decisions.tail(1).initial_maney.values[0]) * 100))

In [2]:
df = pd.read_excel("./database/indicadores petrobras_fase 1_ v1.2.xlsx", sheet_name = "Tendencias").drop(['-','--'], axis=1).dropna()
#profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
#profile

In [3]:
df = df.sort_values(['Data'])
vols = [1,2,3,5,7,9,14,21]
for volatilidade in vols:
    df[["Volatilidade_Close_"+str(volatilidade)]] = df.Fechamento.rolling(volatilidade).std()
    df[["Volatilidade_Min_"+str(volatilidade)]] = df[['Mínimo']].rolling(volatilidade).std()
    df[["Volatilidade_Max_"+str(volatilidade)]] = df[['Máximo']].rolling(volatilidade).std()
    df[["Compra_MACD_"+str(volatilidade)]] = df[['Sinal de Compra (MACD)']].rolling(volatilidade).sum()
    #df[["Compra_MACD_"+str(volatilidade)]] = [1 if not(np.isnan(x)) and int(x) > 0 else 0 for x in df[["Compra_MACD_"+str(volatilidade)]].values]
    df[["Venda_MACD_"+str(volatilidade)]] = df[['Sinal de Venda (MACD)']].rolling(volatilidade).sum()
    #df[["Venda_MACD_"+str(volatilidade)]] = [1 if not(np.isnan(x)) and int(x) > 0 else 0 for x in df[["Venda_MACD_"+str(volatilidade)]].values]
df = df.sort_values(['Data'], ascending=False).fillna(0)

In [54]:
vols = []
compras = []
vendas = []
for col in df.columns:
    if (col.startswith('Volatilidade')):
        vols.append(col)
    if (col.startswith('Compra_MACD_')):
        compras.append(col)
    if (col.startswith('Venda_MACD_')):
        vendas.append(col)



##### Hipótese 1:
Como NB se sai com os decisores individuais?

In [5]:
# Model 1 - Willians
features1 = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','William %R']
labels1 = ['Willians Buy','Willians Sell']
label1 = 'Willians_decisor' 

df = _encode(df, labels1, label1)

# Model 2 - 9MME
features2 = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','Mínima 14','Máxima 14',
             '20MA','12MME','26MME','Oscilação diária','Volatilidade 5MA']
labels2 = ['Posição Compra 9MME','Posição Venda 9MME']
label2 = '9MME_decisor' 

df = _encode(df, labels2, label2)

# Model 3 -Sinais MACD
features3 = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','Mínima 14','Máxima 14',
             '9MME','20MA','12MME','26MME', 'Posição Alta MACD','Posição Baixa MACD',
             'MACD Line','Signal Line','Histograma','Oscilação diária','Volatilidade 5MA']
labels3 = ['Sinal de Compra (MACD)','Sinal de Venda (MACD)']
label3 = 'signalMACD_decisor' 

df = _encode(df, labels3, label3)

# Model 4 - Divergencias
features4 = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','Mínima 14','Máxima 14',
             '9MME','20MA','12MME','26MME',
             'MACD Line','Signal Line','Histograma','Oscilação diária','Volatilidade 5MA',
             'Smin(21 pontos)','Rmax(21 pontos)']
labels4 = ['Tend. Alta por divergência', 'Tend.Baixa por divergência'] #tend alta-buy, baixa-sell
label4 = 'divergency_decisor' 

df = _encode(df, labels4, label4)



print("-----------------------------------------------------------------")
_apply_gaussianNB(df, features1, label1, silent=False)
print("-----------------------------------------------------------------")

_apply_gaussianNB(df, features2, label2, silent=False)

print("-----------------------------------------------------------------")

_apply_gaussianNB(df, features3, label3, silent=False)

print("-----------------------------------------------------------------")

_apply_gaussianNB(df, features4, label4, silent=False)

print("-----------------------------------------------------------------")



-----------------------------------------------------------------
0    304
2    108
1     67
Name: Willians_decisor, dtype: int64
Accuracy (Model: Willians_decisor | Gaussian Naive Bayes): 0.6775407779171895
-----------------------------------------------------------------
0    431
1     24
2     24
Name: 9MME_decisor, dtype: int64
Accuracy (Model: 9MME_decisor | Gaussian Naive Bayes): 0.6725219573400251
-----------------------------------------------------------------
0    446
1     17
2     16
Name: signalMACD_decisor, dtype: int64
Accuracy (Model: signalMACD_decisor | Gaussian Naive Bayes): 0.5545796737766625
-----------------------------------------------------------------
0    472
2      7
Name: divergency_decisor, dtype: int64
Accuracy (Model: divergency_decisor | Gaussian Naive Bayes): 0.383939774153074
-----------------------------------------------------------------


##### * Vamos testar o Complement Naive Bayes, que se adapta melhor para data sets desbalanceados (como no caso de alguns decisores)

In [6]:
# Model 1 - Willians
features1 = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','William %R']
labels1 = ['Willians Buy','Willians Sell']
label1 = 'Willians_decisor' 

df = _encode(df, labels1, label1)

# Model 2 - 9MME
features2 = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','Mínima 14','Máxima 14',
             '20MA','12MME','26MME','Oscilação diária','Volatilidade 5MA']
labels2 = ['Posição Compra 9MME','Posição Venda 9MME']
label2 = '9MME_decisor' 

df = _encode(df, labels2, label2)

# Model 3 -Sinais MACD
features3 = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','Mínima 14','Máxima 14',
             '9MME','20MA','12MME','26MME',
             'MACD Line','Signal Line','Histograma','Oscilação diária','Volatilidade 5MA']
labels3 = ['Sinal de Compra (MACD)','Sinal de Venda (MACD)']
label3 = 'signalMACD_decisor' 

df = _encode(df, labels3, label3)

# Model 4 - Divergencias
features4 = ['Fechamento','Var.Dia (%)','Abertura','Mínimo','Máximo','IBOVESPA','22D ROLLING BETA ',
             'Suporte ','Resistencia','Hammer','Mínima 14','Máxima 14',
             '9MME','20MA','12MME','26MME',
             'MACD Line','Signal Line','Histograma','Oscilação diária','Volatilidade 5MA',
             'Smin(21 pontos)','Rmax(21 pontos)']
labels4 = ['Tend. Alta por divergência', 'Tend.Baixa por divergência'] #tend alta-buy, baixa-sell
label4 = 'divergency_decisor' 

df = _encode(df, labels4, label4)

train = df.loc[df.Data < pd.to_datetime("01/02/2018", format="%d/%m/%Y")]
test = df.loc[df.Data >= pd.to_datetime("01/02/2018", format="%d/%m/%Y")]


print("-----------------------------------------------------------------")
_apply_gaussianNB(df, features1, label1, silent=True)
_apply_complementNB(df, features1, label1, silent=True)
_apply_multinomialNB(df, features1, label1, silent=True)
print("-----------------------------------------------------------------")

_apply_gaussianNB(df, features2, label2, silent=True)
_apply_complementNB(df, features2, label2, silent=True)
_apply_multinomialNB(df, features2, label2, silent=True)
print("-----------------------------------------------------------------")

_apply_gaussianNB(df, features3, label3, silent=True)
_apply_complementNB(df, features3, label3, silent=True)
_apply_multinomialNB(df, features3, label3, silent=True)
print("-----------------------------------------------------------------")

_apply_gaussianNB(df, features4, label4, silent=True)
_apply_complementNB(df, features4, label4, silent=True)
_apply_multinomialNB(df, features4, label4, silent=True)
print("-----------------------------------------------------------------")


-----------------------------------------------------------------
-----------------------------------------------------------------
-----------------------------------------------------------------
-----------------------------------------------------------------
-----------------------------------------------------------------


Ficamos com:

    1 - gaussian
    2 - gaussian
    3 - gaussian
    4 - complement (resposta 1 não existe na base de teste - o alpha=1 contempla o a correção de laplace) - a decisão coube ao indicador em si

In [7]:
model1, scaler = _apply_gaussianNB(df, features1, label1, silent=False)

model2, scaler = _apply_gaussianNB(df, features2, label2, silent=True)

model3, scaler = _apply_gaussianNB(df, features3, label3, silent=True)

model4, scaler = _apply_complementNB(df, features4, label4, silent=True)

labels = [label1, label2, label3, label4]

0    304
2    108
1     67
Name: Willians_decisor, dtype: int64
Accuracy (Model: Willians_decisor | Gaussian Naive Bayes): 0.6775407779171895


In [8]:
from sklearn.base import clone 

def drop_col_feat_imp(model, df, features, label, random_state = 42):
    X_train, y_train, X_test, y_test, scaler = _preprocess(df, features, label)
    # clone the model to have the exact same specification as the one initially trained
    model_clone = clone(model)
    # set random_state for comparability
    model_clone.random_state = random_state
    # training and scoring the benchmark model
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)
    # list for storing feature importances
    importances = []
    # iterating over all columns and storing feature importance (difference between benchmark and new model)
    for col in features:
        model_clone = clone(model)
        model_clone.random_state = random_state
        df2 = df.drop(col, axis=1)
        features2 = copy.deepcopy(features)
        features2.remove(col)
        X_train, y_train, X_test, y_test, scaler = _preprocess(df2, features2, label)
        model_clone.fit(X_train, y_train)
        drop_col_score = model_clone.score(X_train, y_train)
        actual_line = {'col': col, "score":(benchmark_score - drop_col_score), 'label':label}
        importances.append(actual_line)
    return pd.DataFrame(importances)

features = list(df.columns)
for l in labels + labels1 + labels2 + labels3 + labels4 + ['Data', 'Volume']:
    features.remove(l)
# % Willians - all features in
model1, scaler = _apply_gaussianNB(df, features, label1, silent=True)

model2, scaler = _apply_gaussianNB(df, features, label2, silent=True)

model3, scaler = _apply_gaussianNB(df, features, label3, silent=True)

model4, scaler = _apply_complementNB(df, features, label4, silent=True)

labels = [label1, label2, label3, label4]

feature_importances = drop_col_feat_imp(model1, df, features, label1)
feature_importances = feature_importances.append(drop_col_feat_imp(model2, df, features, label2))
feature_importances = feature_importances.append(drop_col_feat_imp(model3, df, features, label3))
feature_importances = feature_importances.append(drop_col_feat_imp(model4, df, features, label4))

feature_importances = feature_importances.query("score > 0")

In [9]:
quant = 100
money_start = test.tail(1).Fechamento.values[0] * quant
tam_maximo_lote = 'Volatilidade 5MA'
coluna_a_operar = 'Fechamento'
wait_time = 3 # para não pegar sinais seguidos em períodos de congestão
    
train, test = _filter_Train_n_Test(df)

In [10]:
help(_encode)

Help on function _encode in module __main__:

_encode(df, labels, label)
    Sempre: 
        0-HODL
        1-BUY
        2-SELL



##### Análise de retornos - para ver o peso de cada decisor
Dado os resultados encontrados, como podemos operar?
100% da banca para todas as operações parece ser arriscado. 
Assim, vamos estabelecer pesos diferentes de acordo com o desempenho individual:

In [38]:
print('Modelo 1 = {}'.format(label1))
decisions_train = pd.DataFrame(columns=['Data','classifier','real_decision','model_decision', 'value_fechamento'])
model1, scaler1 = _apply_gaussianNB(df, features1, label1, silent=True)
decisions_train = _apply_model(train, model1, scaler1, features1, label1, decisions_train)

print('Modelo 2 = {}'.format(label2))
model2, scaler2 = _apply_gaussianNB(df, features2, label2, silent=True)
decisions_train = _apply_model(train, model2, scaler2, features2, label2, decisions_train)

print('Modelo 3 = {}'.format(label3))
model3, scaler3 = _apply_gaussianNB(df, features3, label3, silent=True)
decisions_train = _apply_model(train, model3, scaler3, features3, label3, decisions_train)

print('Modelo 4 = {}'.format(label4))
model4, scaler4 = _apply_complementNB(df, features4, label4, silent=True)
decisions_train = _apply_model(train, model4, scaler4, features4, label4, decisions_train)

Modelo 1 = Willians_decisor
Modelo 2 = 9MME_decisor
Modelo 3 = signalMACD_decisor
Modelo 4 = divergency_decisor


In [39]:
decisions_train.classifier.value_counts()

signalMACD_decisor    59
9MME_decisor          56
Willians_decisor      33
Name: classifier, dtype: int64

In [40]:
help(_apply_model)

Help on function _apply_model in module __main__:

_apply_model(df, model, scaler, features, label, decisions)
    Aplica as decisões dos primeiros modelos, para avaliação de como ponderar para uma gestão de risco baseada em resultados anteriores



In [41]:
help(_apply_decisions)

Help on function _apply_decisions in module __main__:

_apply_decisions(model_decisions, classifier, money_start, decision_weight, n)
    Aqui, aplicamos as decisões dos modelos sem uma gestão de risco, para gerar os pesos de ponderação para a gestão de risco;
    "All in, All out!", ou seja, compramos tudo ou vendemos tudo!



In [42]:
model_decisions_train = decisions_train[['Data',"classifier",'model_decision','value_fechamento']]
model_decisions_train = model_decisions_train.drop_duplicates(subset=['Data','classifier','model_decision'], keep='first')

In [43]:
decision_weight = pd.DataFrame(columns = ['classifier', 'initial_maney', 'final_maney', 'lucro_percentual'])
money_start

1854.0

In [44]:
n = 0
for classifier in model_decisions_train.classifier.drop_duplicates():
    decision_weight = _apply_decisions(
        model_decisions_train.loc[model_decisions_train.classifier == classifier], 
        classifier,
        money_start, 
        decision_weight, n)
    n+=1

--------------------------------------------------------
Willians_decisor
--------------------------------------------------------
Decisão de compra -
Valor: 6.85
Quantidade comprada: 270.6569343065694
Money atual: -2.2737367544323206e-13
Decisão de venda -
Valor: 8.16
Quantidade vendida: 270.6569343065694
Money atual: 2208.5605839416057
Decisão de compra -
Valor: 8.42
Quantidade comprada: 262.29935676266103
Money atual: 0.0
HODL!
Decisão de venda -
Valor: 8.48
Quantidade vendida: 262.29935676266103
Money atual: 2224.2985453473657
Decisão de compra -
Valor: 7.5
Quantidade comprada: 296.57313937964875
Money atual: 0.0
Decisão de venda -
Valor: 8.29
Quantidade vendida: 296.57313937964875
Money atual: 2458.591325457288
HODL!
Decisão de compra -
Valor: 11.75
Quantidade comprada: 209.24181493253514
Money atual: 0.0
HODL!
Decisão de venda -
Valor: 12.62
Quantidade vendida: 209.24181493253514
Money atual: 2640.631704448593
HODL!
Decisão de compra -
Valor: 14.57
Quantidade comprada: 181.237591

In [45]:
decision_weight

,classifier,initial_maney,final_maney,lucro_percentual
0,Willians_decisor,1854.0,2852.143217,0.538373
1,9MME_decisor,1854.0,4601.991935,1.482196
2,signalMACD_decisor,1854.0,3584.04506,0.933142


##### Vamos ponderar a decisão da probabilidade de cada item a partir do lucro percentual encontrado.
o que tiver maior lucro na base de teste, terá maior investimento na decisão; iremos incluir aqui o decisor de divergência (apontou apenas venda na base de teste.)
Imputaremos 60% do lucro percentual total para o mesmo, e criaremos um conjunto de pesos com soma=1 para os decisores:

In [46]:
perc_total = sum(decision_weight.lucro_percentual)

decision_weight.loc[4,'classifier'] = 'divergency_decisor'
decision_weight.loc[4,'lucro_percentual'] = perc_total * 0.6

perc_total = sum(decision_weight.lucro_percentual)

decision_weight.loc[:,'new_weight'] = decision_weight.loc[:,'lucro_percentual'] / perc_total

decision_weight = decision_weight[['classifier','new_weight']]

decision_weight

,classifier,new_weight
0,Willians_decisor,0.113919
1,9MME_decisor,0.31363
2,signalMACD_decisor,0.197451
4,divergency_decisor,0.375


In [47]:
#df = pd.read_excel("./database/indicadores petrobras_fase 1_ v1.2.xlsx", sheet_name = "Tendencias").drop(['-','--'], axis=1).dropna()
train, test = _filter_Train_n_Test(df)
quant = 100
money_start = test.tail(1).Fechamento.values[0] * quant
stop_loss = 0.02
tam_maximo_lote = 'Volatilidade 5MA'
coluna_a_operar = 'Fechamento'
wait_time = 5 # para não pegar sinais seguidos em períodos de congestão


In [48]:
train.head(2)

,Data,Histórico,Fechamento,Var.Dia (%),Var (n),Abertura,Mínimo,Máximo,Volume,IBOVESPA,...,Volatilidade_Close_21,Volatilidade_Min_21,Volatilidade_Max_21,Compra_MACD_21,Venda_MACD_21,Willians_decisor,9MME_decisor,signalMACD_decisor,divergency_decisor,final_decision
819,2018-01-31,17.80,17.80,1.08,0.0108,17.83,17.78,18.00,"744,27M",-0.0068,...,1.035877,0.979804,1.075435,0.0,0.0,0,0,0,0,0
820,2018-01-30,17.61,17.61,-1.81,-0.0181,17.86,17.49,17.86,"815,34M",-0.0051,...,1.031973,0.935257,1.071632,0.0,0.0,0,0,0,0,0


#### Modelos finais para serem utilizados como base:

In [49]:
print('Modelo 1 = {}'.format(label1))
decisions_train = pd.DataFrame(columns=['Data','classifier','real_decision','model_decision', 'value_fechamento'])
model1, scaler1 = _apply_gaussianNB(df, features1, label1, silent=True)
decisions_train = _apply_model(train, model1, scaler1, features1, label1, decisions_train)

print('Modelo 2 = {}'.format(label2))
model2, scaler2 = _apply_gaussianNB(df, features2, label2, silent=True)
decisions_train = _apply_model(train, model2, scaler2, features2, label2, decisions_train)

print('Modelo 3 = {}'.format(label3))
model3, scaler3 = _apply_gaussianNB(df, features3, label3, silent=True)
decisions_train = _apply_model(train, model3, scaler3, features3, label3, decisions_train)

print('Modelo 4 = {}'.format(label4))
model4, scaler4 = _apply_complementNB(df, features4, label4, silent=True)
decisions_train = _apply_model(train, model4, scaler4, features4, label4, decisions_train)


Modelo 1 = Willians_decisor
Modelo 2 = 9MME_decisor
Modelo 3 = signalMACD_decisor
Modelo 4 = divergency_decisor


In [50]:
classifiers = ['Willians_decisor', '9MME_decisor','signalMACD_decisor', 'divergency_decisor']
models = {
    label1:model1,
    label2:model2,
    label3:model3,
    label4:model4
}
scalers = {
    label1:scaler1,
    label2:scaler2,
    label3:scaler3,
    label4:scaler4   
}
features = {
    label1:features1,
    label2:features2,
    label3:features3,
    label4:features4
}

##### E a volatilidade?
Notamos que de acordo com a volatilidade, podemos ter resultados diferentes;

Analisamos, portanto, a volatilidade de períodos diferentes:

In [51]:
vols = [1,2,3,5,7,9,14,21]
maxs = []
roi = []

In [52]:
import warnings
warnings.filterwarnings("ignore")

for _wait_time in vols:
    train, test = _filter_Train_n_Test(df)
    quant = 100
    money_start = test.tail(1).Fechamento.values[0] * quant
    if (_wait_time != 1):
        _max = max(train.Fechamento.rolling(_wait_time).std().dropna())
    else:
        _max = max(train['Oscilação diária'])
    maxs.append(_max)
    decisions = _get_model_decisions(train, classifiers, scalers, models, wait_time=_wait_time, silent_mode=True)
    results = _apply_decisions_withperc(decisions, money_start, silent_mode=True)
    roi.append(results.tail(1).final_maney.values[0])
    print("--------------------------------")
    print("Volatilidade_"+str(_wait_time)+": "+str(_max))
    print("Initial Money: R$"+str(money_start))
    print("Return: "+str(results.tail(1).final_maney.values[0]))

--------------------------------
Volatilidade_1: 1.5697770542341347
Initial Money: R$1854.0
Return: 3451.9329275535474
--------------------------------
Volatilidade_2: 1.569777054234097
Initial Money: R$1854.0
Return: 3518.837121961653
--------------------------------
Volatilidade_3: 1.3054245797185269
Initial Money: R$1854.0
Return: 3505.3582326433834
--------------------------------
Volatilidade_5: 1.286300897923941
Initial Money: R$1854.0
Return: 3376.7618905463983
--------------------------------
Volatilidade_7: 1.2545612019841197
Initial Money: R$1854.0
Return: 3428.797074467895
--------------------------------
Volatilidade_9: 1.186048528143353
Initial Money: R$1854.0
Return: 3318.9350729091557
--------------------------------
Volatilidade_14: 1.3161901813212988
Initial Money: R$1854.0
Return: 3133.7305892512627
--------------------------------
Volatilidade_21: 1.5935409806913836
Initial Money: R$1854.0
Return: 3461.152776288616


#### Vamos focar em 3 dias entre operações

In [55]:
# Modelo final
final_features = ['Willians_decisor' , '9MME_decisor' , 'signalMACD_decisor','divergency_decisor']
final_label = ['final_decision']

df = _final_encode(df, final_features, final_label)

## Dependencia dos modelos criados anteriormente:
classifiers = ['Willians_decisor', '9MME_decisor','signalMACD_decisor', 'divergency_decisor']
models = {
    label1:model1,
    label2:model2,
    label3:model3,
    label4:model4
}
scalers = {
    label1:scaler1,
    label2:scaler2,
    label3:scaler3,
    label4:scaler4   
}
features = {
    label1:features1,
    label2:features2,
    label3:features3,
    label4:features4
}

In [56]:
help(_filter_Train_n_Test)
print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
help(_apply_multinomialNB)

Help on function _filter_Train_n_Test in module __main__:

_filter_Train_n_Test(df)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Help on function _apply_multinomialNB in module __main__:

_apply_multinomialNB(df, features, label, silent=False, _alpha=1)
    Esse algoritmo usa os dados em uma distribuição multinomial, que é uma generalização da distribuição binomial. 
    Essa distribuição é parametrizada por vetores θyi=(θy1,…,θyn), θyi é a probabilidade do evento i ocorrer, dado que a classe é y



In [57]:
quant = 100
money_start = test.tail(1).Fechamento.values[0] * quant
vol_to_buy = -0.1
buy_at_vol = 0.6
_stop_loss = 0 #vai ser sempre o suporte; se bater no suporte, vendemos
_sell_by_stoploss = 0.9
coluna_a_operar = 'Fechamento'
wait_time = 3 # para não pegar sinais seguidos em períodos de congestão
    
train, test = _filter_Train_n_Test(df)
final_features = final_features +vols+compras+vendas
final_model, final_scaler = _apply_multinomialNB(df, final_features, final_label, silent=False)

final_decision
0                 247
1                 232
dtype: int64
Accuracy (Model: ['final_decision'] | Multinomial Naive Bayes): 0.9435382685069009


In [58]:
help(_get_final_decisions)
print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
help(_apply_decisions_withperc)

Help on function _get_final_decisions in module __main__:

_get_final_decisions(df, _stop_loss, _sell_by_stoploss, wait_time, final_scaler, final_model, classifiers, scalers, models)
        -Função final com modelo de trade!
        Iremos, utilizando os suportes como stop loss, e quando encontrar um preço de fechamento 
    igual ou menor ao suporte em um ponto de suporte, a venda de x% (definido pela var _sell_by_stoploss), aplicar os modelos encontrados quando o modelo final decidir agir.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Help on function _apply_decisions_withperc in module __main__:

_apply_decisions_withperc(model_decisions, money_start, silent_mode=True)
    Aplica as decisoes encontradas utilizando a ponderação dos melhores valores como método de gestão de risco



In [59]:
train_decisions = _get_final_decisions(train, _stop_loss, _sell_by_stoploss, wait_time, final_scaler, final_model, classifiers, scalers, models)
train_decisions = _apply_decisions_withperc(train_decisions, money_start, silent_mode=True)
train_decisions.tail(1)

,Data,initial_maney,final_maney,quant_hold,value_fechamento,perc_usado,action
42,2017-12-27 00:00:00,1854.0,3409.384719,0,14.5,0.113919,final


In [60]:
_print_decisions(train_decisions)

Data inicial: 2016-03-03 00:00:00
Dinheiro inicial: R$1854.00
Data final: 2017-12-27 00:00:00
Dinheiro final: R$3409.38
Lucro final : 83.89%


In [61]:
test_decisions = _get_final_decisions(test, _stop_loss, _sell_by_stoploss, wait_time, final_scaler, final_model, classifiers, scalers, models)
test_decisions = _apply_decisions_withperc(test_decisions, money_start, silent_mode=True)
test_decisions.tail(1)

,Data,initial_maney,final_maney,quant_hold,value_fechamento,perc_usado,action
41,2021-02-03 00:00:00,1854.0,2506.079501,0,27.91,0.31137,final


In [62]:
_print_decisions(test_decisions)

Data inicial: 2018-03-19 00:00:00
Dinheiro inicial: R$1854.00
Data final: 2021-02-03 00:00:00
Dinheiro final: R$2506.08
Lucro final : 35.17%
